# Statistical Testing

Structure

- [Notebook Preparation](#notebook-preparation)

- [Mann-Kendall Test](#mann_kendal-testing)

    - [Function & Application](#Function--Application)

- [Station-Centric (micro perspective)](#Station-Centric-(micro-perspective))
    
    Compute Mann–Kendall/Theil-Sen statistcal test estimates on each station-month time series (e.g., Station A in January across years). Then summarize across stations by aggregation into Country / Month or Country groupings by usign medians/means & percent of stations with statisical signifnance.

    This approach analyses the **typical station-month time series** by relevant grouping.

    - [Station-Month time series](#station-month-time-series)
    - [Station-Month time series by Country / Month](#station-month-time-series-by-country--month)
    - [Station-Month time series by Country](#station-month-time-series-by-country)

- [Region-Centric (macro perspective)](#region-centric)

    First aggregate the station-month time series by year into relevant grouping of Country/Month or Country, creating one regional time series per group. Then run Mann–Kendall/Theil-Sen statistcal test estimates on the aggregated group of station-month time series.

    This approach analyses the **average station-month time series** by relevant region groupiing.

    - [Average Slope Per Country & Month](#average-slope-per-country-month) 
    - [Average Slope Per Month](#average-slope-per-month)
    - [Average Slope Per Elevation Band & Month](#Average-Slope-Per-Elevation-Band--Month)


## Notebook Preparation

In [456]:
# Import Packages

import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
import plotly.express as px
import geopandas as gpd
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import pymannkendall as mk
from pathlib import Path
from scipy.stats import norm


In [457]:
# Files & Directories

NB_DIR = Path.cwd()         # Notebook Directory
REPO_ROOT = NB_DIR.parent   # Main Directory

snow_recordings = pd.read_csv(REPO_ROOT/'data/cleaned/snow_recordings.csv')

snow_recordings.head(2)

,id,station_id,year,month,hnsum,winter,name,latitude,longitude,elevation,country,provider,geometry,elevation_band
0,1784526,132,1936,1,37.0,True,Scuol_CH_METEOSWISS,46.79327,10.28324,1303,Switzerland,CH_METEOSWISS,POINT (4342645.189784505 2631133.4129758803),Mid Elevation
1,1618609,153,1936,1,78.0,True,Arosa_CH_METEOSWISS,46.79262,9.67900,1878,Switzerland,CH_METEOSWISS,POINT (4296468.910481082 2631072.9686830333),High Elevation



## Mann_Kendall Testing

A series of Mann_Kendall statistical tests to analyze the changing rate of average snowpack depth across European Weather Stations



- Output 
    - n:   
        - number of years used in the test for that group.

    - trend & p: 
        - result of the Mann–Kendall test — whether a monotonic increase/decrease is statistically significant (p ≤ α).

    - τ (Kendall’s tau): 
        - strength and direction of the monotonic relationship (−1 to +1).

    - Slope (rate of change):
        - Theil–Sen slope (aka Sen’s slope): 
            - estimated rate of change in snow depth in cm/year; multiply by 10 for cm/decade. 
            - Robust to outliers and does not require continuous yearly data.


In [458]:
# Create Dataframe of snow recordings per month 
monthly = (snow_recordings
           .groupby(['country','station_id','year','month'], as_index=False)
           .agg(hnsum=('hnsum','mean')))

monthly

,country,station_id,year,month,hnsum
0,Austria,13,1970,11,5.0
1,Austria,13,1970,12,28.0
2,Austria,13,1971,1,3.0
3,Austria,13,1971,2,77.0
4,Austria,13,1971,3,33.0
...,...,...,...,...,...
318955,Switzerland,2977,2013,2,67.0
318956,Switzerland,2977,2013,3,22.0
318957,Switzerland,2977,2013,4,0.0
318958,Switzerland,2977,2013,5,0.0


### Function & Application

In [459]:
from scipy.stats import theilslopes

# Minimum number of data years require for each Weather Station to be considered for testing
MIN_YEARS = 30

# If no (extreme minimal) rate of change is identified, weather station not utilised.
EPS = 1e-12

# Perform Mann_Kendal test & Theil Slope for monthly Snowpack Recordings
def mk_monthly_with_theil(group_df):
    g = group_df.sort_values('year')
    y = pd.to_numeric(g['hnsum'], errors='coerce').to_numpy()
    x = g['year'].astype(float).to_numpy()
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    n = y.size

    # guards
    if n < MIN_YEARS or np.nanstd(y) <= EPS or np.unique(y).size < 2:
        return {'variant':'skip', 'n_years':n, 'trend':'insufficient',
                'p':np.nan, 'tau':np.nan, 'slope_sen':np.nan,
                'slope_theil':np.nan}

    # MK (Hamed–Rao with Original-Test fallback)
    try:
        res = mk.hamed_rao_modification_test(y)
        if not np.isfinite([res.p, res.Tau, res.slope]).all():
            res = mk.original_test(y); variant = 'original_fallback'
        else:
            variant = 'hamed_rao'
    except Exception:
        res = mk.original_test(y); variant = 'original_fallback'

    # Theil–Sen slope per YEAR (uses actual years)
    try:
        slope_ts, intercept, lo, hi = theilslopes(y, x, 0.95)
    except Exception:
        slope_ts = np.nan

    return {'variant':variant,              # Mann-Kendal Test used
            'n_years':n,                    # Number of data years available from Weather Station 
            'trend':res.trend,              # Monotonic trend 
            'p':res.p,                      # p-value for statistical significance
            'tau':res.Tau,                  # Strength of Monotonic trend
            'slope_sen':res.slope,          # Sen’s (per year)
            'slope_theil':slope_ts}         # Theil–Sen (per year)


In [460]:
# Perform Mann-Kendal test function
rows = []
for (country, station_id, month), group_df in monthly.groupby(['country','station_id','month'], sort=False):
    result = mk_monthly_with_theil(group_df)  # run MK on this station–month series
    result['country'] = country
    result['station_id'] = station_id
    result['month'] = month
    rows.append(result)

per_station_month = pd.DataFrame(rows)

per_station_month['slope_sen_per_decade'] = per_station_month['slope_sen'] * 10

per_station_month['slope_theil_per_decade'] = per_station_month['slope_theil'] * 10

print('Successful Testing \n'
      f'Show .head(1) output previous : \n' 
      f'{per_station_month.head(1)}')


/opt/anaconda3/envs/data_env312/lib/python3.12/site-packages/pymannkendall/pymannkendall.py:99: RuntimeWarning: invalid value encountered in sqrt
  z = (s - 1)/np.sqrt(var_s)
/opt/anaconda3/envs/data_env312/lib/python3.12/site-packages/pymannkendall/pymannkendall.py:103: RuntimeWarning: invalid value encountered in sqrt
  z = (s + 1)/np.sqrt(var_s)
/opt/anaconda3/envs/data_env312/lib/python3.12/site-packages/pymannkendall/pymannkendall.py:103: RuntimeWarning: invalid value encountered in sqrt
  z = (s + 1)/np.sqrt(var_s)
/opt/anaconda3/envs/data_env312/lib/python3.12/site-packages/pymannkendall/pymannkendall.py:103: RuntimeWarning: invalid value encountered in sqrt
  z = (s + 1)/np.sqrt(var_s)
/opt/anaconda3/envs/data_env312/lib/python3.12/site-packages/pymannkendall/pymannkendall.py:99: RuntimeWarning: invalid value encountered in sqrt
  z = (s - 1)/np.sqrt(var_s)
/opt/anaconda3/envs/data_env312/lib/python3.12/site-packages/pymannkendall/pymannkendall.py:103: RuntimeWarning: invalid v

Successful Testing 
Show .head(1) output previous : 
     variant  n_years     trend         p       tau  slope_sen  slope_theil  \
0  hamed_rao       46  no trend  0.088599 -0.173913  -0.333333    -0.333333   

   country  station_id  month  slope_sen_per_decade  slope_theil_per_decade  
0  Austria          13     11             -3.333333               -3.333333  


## Station-Centric (micro perspective)

### Station-Month time series

For all 6 countries, from our Mann-Kendal Testing, we output each station-month time series

A station-centric view of per month slopes

In [461]:
# Geometric data of Weather Stations
sr_meta = snow_recordings[['station_id','geometry','elevation_band']]
sr_meta = (sr_meta
                .drop_duplicates()
                .set_index('station_id')
                [['geometry','elevation_band']]
            )

# Successfully Tested Weather Stations  
tested = per_station_month[per_station_month['trend'].isin(['increasing','decreasing','no trend'])] # Other trend is 'Insufficient'

# Join Geometric data to Dataframe
tested_geo = (tested
                  .join(sr_meta, on='station_id', how='left'))

# Export CSV for visualisations
try:
    station_by_month_tested_path = REPO_ROOT /'Data/Cleaned/Tests/station-month-time-series.csv'
    tested_geo.to_csv(station_by_month_tested_path, index=False)
    if station_by_month_tested_path.exists():
        print(f'Dataframe of Station-Month Time Series \n' 
            f'Number of Successfully Tested Station_by_Month series: {len(tested_geo)}\n'
            f'CSV Exported To: {station_by_month_tested_path} \n')
except Exception as e:
    print('CSV File failed to export \n',e)

print('Dataframe Preview:')
tested_geo.head(5)

Dataframe of Station-Month Time Series 
Number of Successfully Tested Station_by_Month series: 5309
CSV Exported To: /Users/mitchellpalmer/Projects/Europe_Snowpack_Depths/Data/Cleaned/Tests/station-month-time-series.csv 

Dataframe Preview:


,variant,n_years,trend,p,tau,slope_sen,slope_theil,country,station_id,month,slope_sen_per_decade,slope_theil_per_decade,geometry,elevation_band
0,hamed_rao,46,no trend,0.088599,-0.173913,-0.333333,-0.333333,Austria,13,11,-3.333333,-3.333333,POINT (4592485.332991373 2733080.3737341957),Low Elevation
1,hamed_rao,46,no trend,0.894474,0.014493,0.027027,0.027027,Austria,13,12,0.270270,0.270270,POINT (4592485.332991373 2733080.3737341957),Low Elevation
2,hamed_rao,46,no trend,0.293874,0.103382,0.500000,0.500000,Austria,13,1,5.000000,5.000000,POINT (4592485.332991373 2733080.3737341957),Low Elevation
3,hamed_rao,46,no trend,0.160941,0.143961,1.000000,1.000000,Austria,13,2,10.000000,10.000000,POINT (4592485.332991373 2733080.3737341957),Low Elevation
4,hamed_rao,46,no trend,0.337193,0.074396,0.181818,0.181818,Austria,13,3,1.818182,1.818182,POINT (4592485.332991373 2733080.3737341957),Low Elevation


In [462]:
# Function to determine percentage of station-month series are Statistically Significant
def pct_sig(sorted_df): 
    sorted_df = sorted_df.dropna(); 
    return 100 * (sorted_df < 0.05).sum() / max(len(sorted_df),1)

### Station-Month time series by Country & Month

For all 6 countries, here we focus on each **average trend per station in each month of each Country.** 

A station-centric or **average of stations per month slopes view**

In [463]:
# tested data preparation
tested_all = tested.copy()

# Datafame of Country_by_Month trend analysis
country_month = (tested_all.groupby(['country','month'])
    .agg(num_stations=('station_id','nunique'),                             # Number of stations' data 
         median_years=('n_years','median'),                                 # Median number of year for contributin station data
         percent_sig=('p', pct_sig),                                        # Percent of contributin station data that are Statistically Significant
         median_slope_theil_per_year=('slope_theil','median'),              # Median Theil Slope Per Year
         median_slope_theil_per_decade=('slope_theil_per_decade','median'), # Median Theil Slope Per Decade
         mean_slope_theil_per_year=('slope_theil','mean'),                  # Mean Theil Slope Per Year
         mean_slope_theil_per_decade=('slope_theil_per_decade','mean'))     # Mean Theil Slope Per Decade
    .reset_index()
    .sort_values(['country','month']))

cols = ['percent_sig','median_slope_theil_per_year','mean_slope_theil_per_year',
        'median_slope_theil_per_decade','mean_slope_theil_per_decade']

# Export CSV for visualisations
try:
    country_by_month_path = REPO_ROOT /'Data/Cleaned/Tests/station-month-time-series-by-country-month.csv'
    country_month.to_csv(country_by_month_path, index=False)
    if country_by_month_path.exists():
        print(f'Dataframe of Countries by Month snowpack analysis : {country_by_month_path.name}\n' 
            f'CSV Exported To: {country_by_month_path} \n')
            
except Exception as e:
    print('CSV File failed to export \n',e)

print(f'Dataframe Preview: \n')
country_month.sort_values(['country','month'],ascending=True).head(5)


Dataframe of Countries by Month snowpack analysis : station-month-time-series-by-country-month.csv
CSV Exported To: /Users/mitchellpalmer/Projects/Europe_Snowpack_Depths/Data/Cleaned/Tests/station-month-time-series-by-country-month.csv 

Dataframe Preview: 



,country,month,num_stations,median_years,percent_sig,median_slope_theil_per_year,median_slope_theil_per_decade,mean_slope_theil_per_year,mean_slope_theil_per_decade
0,Austria,1,321,46.0,5.607477,-0.125000,-1.250000,-0.133281,-1.332814
1,Austria,2,321,46.0,5.919003,0.142857,1.428571,0.267544,2.675437
2,Austria,3,321,46.0,4.361371,-0.074074,-0.740741,-0.125828,-1.258282
3,Austria,4,321,46.0,44.236760,-0.121212,-1.212121,-0.281902,-2.819025
4,Austria,5,269,46.0,10.780669,0.000000,0.000000,0.001513,0.015127


**Example Result Interpretation**

Austria - Based on 321 station-month time series (median record length 46 years) using the Mann–Kendall (Hamed–Rao autocorrelation correction) test, the median station-level trend in mean snowpack for month 1 (January) is -0.125 cm/year (-1.25 cm/decade). The mean station-level trend for January is -0.133 cm/year (-1.33 cm/decade). 5.6% of station-month series in Janurary show a statistically significant monotonic trend. 

This station-centric analysis use all stations as weighted equally; not area-weighted.

### Station-Month time series by Country

For all 6 countries, here we focus on each **average trend per station_month series within each Country.** 

A station-centric or **average-of-station-month slopes view**

In [ ]:
# Dataframe of each Country's average station_month series
country_overall = (tested_all.groupby('country')
                   .agg(n_station_month_series=('station_id','count'),                        # count of station-month series
                        n_stations=('station_id','nunique'),                                    # Number unique stations per country
                        median_years=('n_years','median'),                                      # Median number of years in station_month series
                        percent_sig_station_month_series=('p', pct_sig),                        # percentage of station_month series that are statistically significant 
                        median_station_slope_cm_per_year=('slope_theil','median'),              # Median slope per year
                        median_station_slope_cm_per_decade=('slope_theil_per_decade','median'), # Median slope per decade
                        mean_station_slope_cm_per_year=('slope_theil','mean'),                  # Mean slope per year
                        mean_station_slope_cm_per_decade=('slope_theil_per_decade','mean'))     # Mean slope per decade
                   .reset_index())

# Export CSV for visualisations
try:
    country_station_month_path = REPO_ROOT /'Data/Cleaned/Tests/station-month-time-series-by-country.csv'
    country_overall.to_csv(country_station_month_path, index=False)
    if country_station_month_path.exists():
        print(f'Dataframe of each Country"s average station_month series of snowpack depth trends: {country_station_month_path.name}\n' 
            f'CSV Exported To: {country_station_month_path} \n')
            
except Exception as e:
    print('CSV File failed to export \n',e)

print(f'Dataframe Preview: \n')
country_overall



Dataframe of each Country"s average station_month series of snowpack depth trends: country_by_station-month_time_series.csv
CSV Exported To: /Users/mitchellpalmer/Projects/Europe_Snowpack_Depths/Data/Cleaned/Tests/country_by_station-month_time_series.csv 

Dataframe Preview: 



,country,n_station_month_series,n_stations,median_years,percent_sig_station_month_series,median_station_slope_cm_per_year,median_station_slope_cm_per_decade,mean_station_slope_cm_per_year,mean_station_slope_cm_per_decade
0,Austria,2195,321,46.0,17.494305,-0.035714,-0.357143,-0.117927,-1.179269
1,France,300,46,50.0,10.000000,-0.133019,-1.330189,-0.174912,-1.749122
2,Germany,1103,159,53.0,14.868540,-0.026316,-0.263158,-0.107828,-1.078278
3,Italy,249,53,35.0,11.646586,0.000000,0.000000,-0.022740,-0.227403
4,Slovenia,366,53,56.0,33.333333,-0.087605,-0.876050,-0.144193,-1.441927
5,Switzerland,1096,163,59.0,12.956204,-0.040221,-0.402206,-0.126877,-1.268768


**Example Result Interpretation**

Austria — Based on 2,195 station–month series (median record length 46 years) using the Mann–Kendall (Hamed–Rao autocorrelation correction) test, the median station-level trend in mean snowpack is −0.036 cm/year (−0.357 cm/decade). The mean station-level trend is −0.118 cm/year (−1.179 cm/decade). 17.5% of station–month series show a statistically significant monotonic trend (α=0.05).

This station-centric analysis use all stations as weighted equally; not area-weighted.

### Station_by_year

Across all 6 countries, our MK Testing focuses here on each Station's annual trend by averaging winter months

In [465]:
# Stouffer's method combines p-values by transforming each p-value into a Z-score, 
# summing these Z-scores, and then converting the sum back to a combined p-value using the normal distribution. 
    # Assume p-value independence
def stouffer_p(ps, signs=None, weights=None):
    ps = np.asarray(ps, float)
    ok = np.isfinite(ps) & (ps > 0) & (ps <= 1)
    ps = ps[ok]
    if ps.size == 0: 
        return np.nan
    z = norm.isf(ps / 2.0)                 # two-sided p -> Z
    if signs is not None:
        signs = np.asarray(signs, float)[ok]
        z = z * np.sign(signs)             # give Z a direction
    if weights is None:
        Z = z.sum() / np.sqrt(len(z))
    else:
        w = np.asarray(weights, float)[ok]
        Z = (w * z).sum() / np.sqrt((w ** 2).sum())
    return 2 * norm.sf(abs(Z))             # combined two-sided p


In [466]:
# weights: give more weight to longer time series
per_station_sig = (tested_all
    .groupby(['country','station_id'], as_index=False)
    .apply(lambda g: pd.Series({
        'n_months'        : g['month'].nunique(),
        'pct_sig_months'  : 100 * (g['p'] <= 0.05).mean(),
        'min_p'           : g['p'].min(),
        'p_combined'      : stouffer_p(g['p'].values,
                                       signs=np.sign(g['slope_theil'].values),
                                       weights=np.sqrt(g['n_years'].values),)
    }))
    .reset_index(drop=True)
)
per_station_sig

/var/folders/bs/_y1b6rb96rb21tm4p3r5fpz80000gn/T/ipykernel_41690/2939328329.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,country,station_id,n_months,pct_sig_months,min_p,p_combined
0,Austria,13,6.0,0.000000,0.075116,0.452272
1,Austria,20,6.0,0.000000,0.292314,0.807434
2,Austria,31,6.0,0.000000,0.056804,0.507123
3,Austria,43,7.0,28.571429,0.014137,0.064821
4,Austria,50,7.0,14.285714,0.029160,0.185393
...,...,...,...,...,...,...
790,Switzerland,2929,7.0,0.000000,0.102281,0.620208
791,Switzerland,2937,7.0,0.000000,0.142795,0.013715
792,Switzerland,2941,7.0,28.571429,0.044390,0.005337
793,Switzerland,2967,7.0,0.000000,0.122109,0.932954


In [ ]:
# Dataframe of each successfully tested Weather Station's average snowpack change
per_station_avg = (tested_all.groupby(['country','station_id'])
                   .agg(n_months=('month','nunique'),
                        median_slope_theil_per_year=('slope_theil','median'),
                        median_slope_theil_per_decade=('slope_theil_per_decade','median'),
                        mean_slope_theil_per_year=('slope_theil','mean'),
                        mean_slope_theil_per_decade=('slope_theil_per_decade','mean'))
                   .reset_index())

# Addition of p_combined value 
pss = per_station_sig[['station_id','p_combined']].set_index('station_id')
per_station_avg = per_station_avg.join(pss, on='station_id', how='inner')

# Export CSV File for visualisations
try:
    per_station_avg_path = REPO_ROOT /'Data/Cleaned/Tests/per_station_series.csv'
    per_station_avg.to_csv(per_station_avg_path)
    if per_station_avg_path.exists():
        print(f'Dataframe of individual Weather Stations average annual snowpack slopes changes \n' 
            f'Number of Successfully Tested Stations: {len(per_station_avg)}\n'
            f'CSV Exported To: {per_station_avg_path}')
except Exception as e:
    print('Dataframe failed to export',e)


per_station_avg.head(5)

Dataframe of individual Weather Stations average annual snowpack slopes changes 
Number of Successfully Tested Stations: 795
CSV Exported To: /Users/mitchellpalmer/Projects/Europe_Snowpack_Depths/Data/Cleaned/Tests/per_station_snowpack_avg.csv


,country,station_id,n_months,median_slope_theil_per_year,median_slope_theil_per_decade,mean_slope_theil_per_year,mean_slope_theil_per_decade,p_combined
0,Austria,13,6,0.104423,1.044226,0.229252,2.292520,0.452272
1,Austria,20,6,0.000000,0.000000,0.015931,0.159314,0.807434
2,Austria,31,6,0.000000,0.000000,0.050926,0.509259,0.507123
3,Austria,43,7,-0.166667,-1.666667,-0.232876,-2.328764,0.064821
4,Austria,50,7,-0.052632,-0.526316,-0.117846,-1.178459,0.185393


## Region-Centric (marco perspective)

### Average Slope Per Country Month

In [468]:
MIN_YEARS = 30
MIN_STATIONS_PER_YEAR = 10  # coverage guardrail

# 1) Build a country–year–month aggregate series (mean across stations that year)
cm = (snow_recordings
      .groupby(['country','year','month'])
      .agg(n_stations=('station_id','nunique'),
           hn_mean=('hnsum','mean'))          # or weighted mean, see below
      .reset_index())

# require a minimum number of stations in a given year-month to avoid 1-station years
cm = cm[cm['n_stations'] >= MIN_STATIONS_PER_YEAR]

# 2) MK/Hamed–Rao on each country–month aggregated time series
def mk_country_month(group_df: pd.DataFrame) -> pd.Series:
    """
    Run MK on one (country, month) series of yearly values.

    Expects group_df to have at least: 'year', 'h'
    Optionally: 'n_stations' = # stations used for that (country, month, year)
    """
    # 1) Sort by year and build the time series
    group_df = group_df.sort_values('year')
    y = group_df['hn_mean'].astype(float).to_numpy()
    ok = np.isfinite(y)
    y = y[ok]
    n_years = len(y)

    # 2) Stations coverage summary (if available)
    stations_median = stations_min = stations_max = np.nan
    if 'n_stations' in group_df.columns:
        stations = group_df.loc[ok, 'n_stations'].to_numpy()
        if stations.size:
            stations_median = int(np.median(stations))
            stations_min    = int(stations.min())
            stations_max    = int(stations.max())

    # 3) Too few years? Return an "insufficient" record
    if n_years < MIN_YEARS:
        return pd.Series({
            'n_years': n_years,
            'stations_median_per_year': stations_median,
            'stations_min_per_year': stations_min,
            'stations_max_per_year': stations_max,
            'p': np.nan,
            'tau': np.nan,
            'slope_per_year': np.nan,
            'slope_per_decade': np.nan,
            'variant': 'insufficient',
        })

    # 4) MK (Hamed–Rao) with fallback to original
    try:
        results = mk.hamed_rao_modification_test(y)
        variant = 'hamed_rao'
    except Exception:
        results = mk.original_test(y)
        variant = 'original'

    # 5) Assemble result row for this (country, month)
    return pd.Series({
        'n_years': n_years,
        'stations_median_per_year': stations_median,
        'trend':results.trend,
        'p': results.p,
        'tau': results.Tau,
        'slope_per_year': results.slope,          # cm/year
        'slope_per_decade': results.slope * 10,   # cm/decade
        'variant': variant,
    })

country_month_macro = (
    cm.groupby(['country', 'month'], group_keys=False)
      .apply(mk_country_month)
      .reset_index()
)


try:
    country_month_macro_path = REPO_ROOT / 'Data/Cleaned/Tests/avg_country_month_trends.csv'
    country_month_macro.to_csv(country_month_macro_path)
    if country_month_macro_path.exists():
        print(f'Average Snowpack Depth Trend Per Country & Month : {country_month_macro_path.name} \n'
              f'Dataframe Exported To: {country_month_macro_path} \n')
except Exception as e:
    print('Dataframe Failed to Export \n')

print('Dataframe Preview: \n')
country_month_macro.head(5)

Average Snowpack Depth Trend Per Country & Month : avg_country_month_trends.csv 
Dataframe Exported To: /Users/mitchellpalmer/Projects/Europe_Snowpack_Depths/Data/Cleaned/Tests/avg_country_month_trends.csv 

Dataframe Preview: 



/var/folders/bs/_y1b6rb96rb21tm4p3r5fpz80000gn/T/ipykernel_41690/45724395.py:74: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(mk_country_month)


,country,month,n_years,stations_median_per_year,trend,p,tau,slope_per_year,slope_per_decade,variant
0,Austria,1,46,362,no trend,0.761626,-0.041546,-0.112497,-1.124971,hamed_rao
1,Austria,2,46,362,no trend,0.352503,0.103382,0.304458,3.044580,hamed_rao
2,Austria,3,46,362,no trend,0.663174,-0.045411,-0.101500,-1.015003,hamed_rao
3,Austria,4,46,362,decreasing,0.000767,-0.265700,-0.382712,-3.827118,hamed_rao
4,Austria,5,46,362,no trend,0.365252,-0.105314,-0.015908,-0.159083,hamed_rao


### Average Slope Per Month

In [469]:
MIN_YEARS = 30
MIN_STATIONS_PER_YEAR = 10  # coverage guardrail

# 1) Build a year–month aggregate series (mean across stations that year for all countries)
cm_month = (snow_recordings
      .groupby(['year','month'])
      .agg(n_stations=('station_id','nunique'),
           hn_mean=('hnsum','mean'))          # or weighted mean, see below
      .reset_index())

cm_month = cm_month[cm_month['n_stations'] >= MIN_STATIONS_PER_YEAR]

# 2)

def mk_month(group_df: pd.DataFrame) -> pd.Series:
    """ 
    Run MK on one (month) series of yearly values.

    Expects group_df to have at least: 'year', 'hn_mean' 
    """ 
    # 1) Sort by year and build the time series
    group_df = group_df.sort_values('year')
    y = group_df['hn_mean'].astype(float).to_numpy()
    mask = np.isfinite(y)          # <— True where y is a real number; False for NaN, +inf, -inf
    y = y[mask]                    # keep only the “True” values in the Mask
    n_years = len(y)

    # 2) Station coverage summary (if availabile)
    stations_median = np.nan
    if 'n_stations' in group_df.columns:
        stations = group_df.loc[mask, 'n_stations'].to_numpy()
        if stations.size:
            stations_median = int(np.median(stations))

    # 3) Too few years? Return an "insufficient" record
    if n_years < MIN_YEARS:
        return pd.Series({
            'n_years': n_years,
            'stations_median_per_year': stations_median,
            'p': np.nan,
            'tau': np.nan,
            'slope_per_year': np.nan,
            'slope_per_decade': np.nan,
            'variant': 'insufficient',
        }) 
    
    # 4) MK (Hamed-Rao) with fallback to original

    try:
        result = mk.hamed_rao_modification_test(y)
        variant = 'hamed_rao'
    except Exception:
        result = mk.original_test(y)
        variant = 'original'

    # 5) Assemble result row for this (month)
    return pd.Series({
        'n_years': n_years,
        'stations_median_per_year': stations_median,
        'trend':result.trend,
        'p': result.p,
        'tau': result.Tau,
        'slope_per_year': result.slope,          # cm/year
        'slope_per_decade': result.slope * 10,   # cm/decade
        'variant': variant,
    }) 

month_macro = (
    cm_month.groupby('month', group_keys=False)
    .apply(mk_month)
    .reset_index() 
)

try:
    month_macro_path = REPO_ROOT / 'Data/Cleaned/Tests/avg_month_trends.csv'
    month_macro.to_csv(month_macro_path)
    if month_macro_path.exists():
        print(f'Average Snowpack Depth Trend Per Month : {month_macro_path.name} \n'
              f'Dataframe Exported To: {month_macro_path} \n')
except Exception as e:
    print('Dataframe Failed to Export \n')

print('Dataframe Preview: \n')
month_macro.head(5)


Average Snowpack Depth Trend Per Month : avg_month_trends.csv 
Dataframe Exported To: /Users/mitchellpalmer/Projects/Europe_Snowpack_Depths/Data/Cleaned/Tests/avg_month_trends.csv 

Dataframe Preview: 



/var/folders/bs/_y1b6rb96rb21tm4p3r5fpz80000gn/T/ipykernel_41690/1370534260.py:70: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(mk_month)


,month,n_years,stations_median_per_year,trend,p,tau,slope_per_year,slope_per_decade,variant
0,1,84,702,no trend,0.276705,-0.089501,-0.184386,-1.843855,hamed_rao
1,2,84,704,no trend,0.096924,-0.109581,-0.191993,-1.919928,hamed_rao
2,3,84,696,no trend,0.576304,-0.033276,-0.056732,-0.567322,hamed_rao
3,4,84,682,decreasing,0.039492,-0.153184,-0.165556,-1.655558,hamed_rao
4,5,84,606,decreasing,0.000118,-0.348824,-0.109625,-1.096252,hamed_rao


### Average Slope Per Elevation Band & Month

In [470]:

#1 ) Build an Elevation Band aggregate series (mean across stations that year for all Elevation Bands)
cm_elevation = (snow_recordings
                .groupby(['year','month','elevation_band'])
                .agg(
                    n_stations=('station_id','nunique'),
                    hn_mean=('hnsum','mean')
                    ) 
                .reset_index()
                )

# Filter for Number of Stations threshold requirement
cm_elevation = cm_elevation[cm_elevation['n_stations'] >= MIN_STATIONS_PER_YEAR]

# 2) Build MK Test function for Elevation

def mk_elevation(group_df: pd.DataFrame) -> pd.Series:
    """ 
    Run MK on one elevation band series of yearly values. 

    Expect group_df to have at least: 'year' , 'hn_mean'
    """

    # 1) Sort by year and build the time series
    group_df = group_df.sort_values('year')
    y = group_df['hn_mean'].astype(float).to_numpy()
    mask = np.isfinite(y)          # <— True where y is a real number; False for NaN, +inf, -inf
    y = y[mask]                    # keep only the “ok” values
    n_years = len(y) 

    # 2) Station coverage summary (if availabile)
    stations_median = np.nan
    if 'n_stations' in group_df.columns:
        stations = group_df.loc[mask, 'n_stations'].to_numpy()
        if stations.size:
            stations_median = int(np.median(stations))

    # 3) Too few years? Return an "insufficient" record
    if n_years < MIN_YEARS:
        return pd.Series({
            'n_years': n_years,
            'stations_median_per_year': stations_median,
            'p': np.nan,
            'tau': np.nan,
            'slope_per_year': np.nan,
            'slope_per_decade': np.nan,
            'variant': 'insufficient',
        }) 
    
    # 4) MK (Hamed-Rao) with fallback to original MK

    try:
        result = mk.hamed_rao_modification_test(y)
        variant = 'hamed_rao'
    except Exception:
        result = mk.original_test(y)
        variant = 'original' 
    
    # 5) Assubme result row for this test (Elevation Band) 
    return pd.Series({
        'n_years': n_years,
        'stations_median_per_year': stations_median,
        'trend':result.trend,
        'p': result.p,
        'tau': result.Tau,
        'slope_per_year': result.slope,          # cm/year
        'slope_per_decade': result.slope * 10,   # cm/decade
        'variant': variant,
    }) 

elevation_macro = (
    cm_elevation.groupby(['month','elevation_band'], group_keys= False)
    .apply(mk_elevation)
    .reset_index()
)

try:
    elevation_macro_path = REPO_ROOT / 'Data/Cleaned/Tests/avg_elevation_month_trends.csv'
    elevation_macro.to_csv(elevation_macro_path)
    if elevation_macro_path.exists():
        print(f'Average Snowpack Depth Trend Per Elevation Band & Month : {elevation_macro_path.name} \n'
              f'Dataframe Exported To: {elevation_macro_path} \n')
except Exception as e:
    print('Dataframe Failed to Export \n')

print('Dataframe Preview: \n')
elevation_macro.head(5)


Average Snowpack Depth Trend Per Elevation Band & Month : avg_elevation_month_trends.csv 
Dataframe Exported To: /Users/mitchellpalmer/Projects/Europe_Snowpack_Depths/Data/Cleaned/Tests/avg_elevation_month_trends.csv 

Dataframe Preview: 



/var/folders/bs/_y1b6rb96rb21tm4p3r5fpz80000gn/T/ipykernel_41690/2464781839.py:72: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(mk_elevation)


,month,elevation_band,n_years,stations_median_per_year,trend,p,tau,slope_per_year,slope_per_decade,variant
0,1,High Elevation,84,145,no trend,0.338525,-0.058520,-0.169975,-1.699746,hamed_rao
1,1,Low Elevation,73,231,no trend,0.186999,-0.116438,-0.177586,-1.775862,hamed_rao
2,1,Mid Elevation,82,363,no trend,0.108624,-0.129178,-0.293419,-2.934193,hamed_rao
3,2,High Elevation,84,146,decreasing,0.026584,-0.128514,-0.317632,-3.176317,hamed_rao
4,2,Low Elevation,73,227,no trend,0.360313,-0.076104,-0.105305,-1.053053,hamed_rao
